# Ajuste de hiperparámetros y flujos de trabajo en Machine Learning 
Actividad Lección 11 || Programación Python para Machine Learning

Objetivos:
* Conocer los motivos fundamentales por los que es necesario llevar a cabo un proceso de ajuste de parámetros en modelos supervisados de Machine Learning.
* Dominar las técnicas de implementación de los métodos más comunes de ajuste de hiperparámetros en Python.
* Describir qué es un flujo de trabajo en Machine Learning y su utilidad.
* Aprender a utilizar las técnicas de implementación de flujos de trabajo en Machine Learning en Python.

Datos del alumno:
* Víctor Luque Martín
* Máster Avanzado en Programación en Python para Hacking, BigData y Machine Learning

Fecha: 13/01/2023

# Tabla de Contenidos
1. [Importes](#importes)
2. [Carga de datos](#carga)
3. [Análisis de los datos](#analisis)
3. [Flujo de trabajo](#pipeline)
4. [Ajuste de hiperparámetros](#ajuste)
5. [Conclusiones](#conclusiones)

# Importes <a name="importes"></a>

In [3]:
import random, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import (
    RepeatedStratifiedKFold,
    GridSearchCV,
)
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Carga de datos <a class="anchor" name="carga"></a>
Se carga el dataset [Census Income](https://archive.ics.uci.edu/ml/datasets/Census+Income) para realizar el ajuste de hiperparámetros y el flujo de parámetros

In [4]:
filename = "dermatology.data"
col_names = ["erythema", "scaling", "definite borders", "itching", "koebner phenomenon",
             "polygonal papules", "follicular papules", "oral mucosal involvement", "knee and elbow involvement",
             "scalp involvement", "family history", "melanin incontinence", "eosinophils in the infiltrate",
             "PNL infiltrate", "fibrosis of the papillary dermis", "exocytosis", "acanthosis", "hyperkeratosis",
             "parakeratosis", "clubbing of the rete ridges", "elongation of the rete ridges", 
             "thinning of the suprapapillary epidermis", "spongiform pustule", "munro microabcess", "focal hypergranulosis", 
             "disappearance of the granular layer", "vacuolisation and damage of basal layer", "spongiosis", 
             "saw-tooth appearance of retes", "follicular horn plug", "perifollicular parakeratosis", 
             "inflammatory monoluclear inflitrate", "band-like infiltrate", "age", "class"]
col_names = [x.replace(" ", "_").lower() for x in col_names]
df = pd.read_csv(filename, names=col_names)
df.shape

(366, 35)

In [5]:
df.head()

,erythema,scaling,definite_borders,itching,koebner_phenomenon,polygonal_papules,follicular_papules,oral_mucosal_involvement,knee_and_elbow_involvement,scalp_involvement,...,disappearance_of_the_granular_layer,vacuolisation_and_damage_of_basal_layer,spongiosis,saw-tooth_appearance_of_retes,follicular_horn_plug,perifollicular_parakeratosis,inflammatory_monoluclear_inflitrate,band-like_infiltrate,age,class
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45,3


# Análisis de los datos <a class="anchor" name="analisis"></a>
A continuación se muestran las clases que contienen valores perdidos:

In [6]:
df[df == '?'] = np.nan
df.isna().sum()[df.isna().sum() > 0]

age    8
dtype: int64

Comprobaremos si el dataset está balanceado mostrando el número de elementos por cada clase:

In [7]:
df["class"].value_counts()

1    112
3     72
2     61
5     52
4     49
6     20
Name: class, dtype: int64

Como podemos observar, el dataset está desbalanceado, contiene valores perdidos en la columna de la edad y todas sus variables son numéricas.

# Flujo de trabajo <a class="anchor" name="pipeline"></a>
Se crea un flujo de trabajo para el modelo de regresión logística con las siguientes fases:
1. Imputación de valores perdidos por la mediana.
2. Escalado de datos utilizando StandardScaler.
3. Algoritmo a utilizar: MLPClassifier.

In [8]:
X = df.drop(columns=['class'])
y = df['class']
seed = random.seed(time.time())

pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('mlp', MLPClassifier(random_state=seed))
])

space = [{
    'mlp__hidden_layer_sizes': [(10,), (50,), (100,), 
                                (10, 10), (50, 50), (100, 100), 
                                (10, 10, 10), (50, 50, 50), (100, 100, 100)],
    'mlp__activation': ['logistic', 'tanh', 'relu'],
    'mlp__solver': ['lbfgs', 'adam'],
    'mlp__learning_rate': ['constant', 'invscaling', 'adaptive'],
    'mlp__max_iter': [1000, 2000, 3000, 4000, 5000],
    'mlp__learning_rate_init': [1e-4, 1e-3, 1e-2, 1e-1, 1],
}]

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)

# Ajuste de hiperparámetros <a class="anchor" name="ajuste"></a>
Haciendo uso de GridSearchCV del módulo `model_selection` de `sklearn` se ajustan los hiperparámetros del modelo de la red neuronal.

In [10]:
grid = GridSearchCV(pipe, space, scoring='balanced_accuracy', n_jobs=-1, cv=cv)
result = grid.fit(X, y)
print(f"Best Score: {result.best_score_}")
print(f"Best Params: {result.best_params_}")

Best Score: 0.9737321937321937
Best Params: {'mlp__activation': 'relu', 'mlp__hidden_layer_sizes': (50, 50), 'mlp__learning_rate': 'constant', 'mlp__learning_rate_init': 0.0001, 'mlp__max_iter': 5000, 'mlp__solver': 'adam'}


# Conclusiones <a class="anchor" name="conclusiones"></a>
Tras el ajuste de hiperparámetros se obtiene un mejor resultado del modelo de red neuronal.<br>
El mejor resultado obtenido es de un `0.9737321937321937` de precisión balanceada.<br>
Los mejores hiperparámetros son los siguientes:
* `hidden_layer_sizes`: (50, 50)
* `activation`: relu
* `solver`: adam
* `learning_rate`: constant
* `learning_rate_init`: 0.0001
* `max_iter`: 5000